In [1]:
#import libraries 
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType, DateType
from pyspark.sql.functions import desc, col, lower, when, avg, sum, length
from google.cloud import storage

In [94]:
spark.stop()

# Create Spark Session

In [2]:
#create spark session 
spark = SparkSession.builder\
        .appName("Spotify-spark")\
        .master("local[*]")\
        .config("spark.jars", "/home/rihab/Projects/spark/notebooks/gcs-connector-hadoop3-2.2.9-shaded.jar") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable", True) \
        .config("spark.sql.repl.eagerEval.enabled", True) \
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/rihab/Downloads/golden-system-411709-d84051b4e799.json") \
        .getOrCreate()

24/07/15 00:07:52 WARN Utils: Your hostname, rihab-X556UJ resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp3s0)
24/07/15 00:07:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/15 00:07:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [102]:
print(spark.sparkContext._jsc.sc().listJars())

Vector(spark://192.168.1.10:38439/jars/gcs-connector-hadoop3-2.2.9-shaded.jar)
24/07/14 20:26:31 WARN TransportChannelHandler: Exception in connection from /192.168.1.10:45176
java.lang.IllegalArgumentException: Too large frame: 5135603447292250188
	at org.sparkproject.guava.base.Preconditions.checkArgument(Preconditions.java:119)
	at org.apache.spark.network.util.TransportFrameDecoder.decodeNext(TransportFrameDecoder.java:148)
	at org.apache.spark.network.util.TransportFrameDecoder.channelRead(TransportFrameDecoder.java:98)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.channelRead(DefaultChannelPipeline.java:1410)
	at io.netty.channel.AbstractChannelHandle

In [97]:
#Check the catalog that should countain the list of names of databases, tables and columns 
spark.catalog.listTables()

[]

In [98]:
#These are the configurations of the pyspark cluster
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.sql.repl.eagerEval.enabled', 'True'),
 ('spark.sql.warehouse.dir',
  'file:/home/rihab/Projects/spark/notebooks/spark-warehouse'),
 ('spark.hadoop.fs.AbstractFileSystem.gs.impl',
  'com.google.cloud.hadoop.fs.gcs.GoogleHadoo

# Read input data

In [9]:
schema = StructType([
    StructField("_c0", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("rank", IntegerType(), True),
    StructField("artist", StringType(), True),
    StructField("url", StringType(), True),
    StructField("region", StringType(), True),
    StructField("chart", StringType(), True),
    StructField("trend", StringType(), True),
    StructField("streams", IntegerType(), True),
    StructField("track_id", StringType(), True),
    StructField("album", StringType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("explicit", BooleanType(), True),
    StructField("release_date", DateType(), True),
    StructField("available_markets", StringType(), True),
    StructField("af_danceability", FloatType(), True),
    StructField("af_energy", FloatType(), True),
    StructField("af_key", IntegerType(), True),
    StructField("af_loudness", FloatType(), True),
    StructField("af_mode", IntegerType(), True),
    StructField("af_speechiness", FloatType(), True),
    StructField("af_acousticness", FloatType(), True),
    StructField("af_instrumentalness", FloatType(), True),
    StructField("af_liveness", FloatType(), True),
    StructField("af_valence", FloatType(), True),
    StructField("af_tempo", FloatType(), True),
    StructField("af_time_signature", IntegerType(), True)
])

In [10]:
data = spark.read.csv('../data/merged_data_10k.csv', 
                      schema=schema,
                     sep=",")
display(data)

24/07/15 00:20:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


_c0,title,rank,artist,url,region,chart,trend,streams,track_id,album,popularity,duration_ms,explicit,release_date,available_markets,af_danceability,af_energy,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_valence,af_tempo,af_time_signature
null,title,null,date,artist,url,region,chart,null,streams,track_id,null,null,null,null,release_date,null,null,null,null,null,null,null,null,null,null,null,null
0,Chantaje (feat. M...,1,2017-01-01,Shakira,https://open.spot...,Argentina,top200,null,253019.0,6mICuAdrwEjh6Y6lr...,null,null,null,null,2017-05-26,null,0.852,null,8.0,null,0.0,0.0776,0.187,3.05E-5,0.159,0.907,null
1,Vente Pa' Ca (fea...,2,2017-01-01,Ricky Martin,https://open.spot...,Argentina,top200,null,223988.0,7DM4BPaS7uofFul3y...,null,null,null,null,2016-09-22,null,0.663,null,11.0,null,0.0,0.226,0.00431,1.69E-5,0.101,0.533,null
2,Reggaetón Lento (...,3,2017-01-01,CNCO,https://open.spot...,Argentina,top200,null,210943.0,3AEZUABDXNtecAOSC...,null,null,null,null,2016-08-26,null,0.761,null,4.0,null,0.0,0.0502,0.4,0.0,0.176,0.71,null
3,Safari,4,2017-01-01,"J Balvin, Pharrel...",https://open.spot...,Argentina,top200,null,173865.0,6rQSrBHf7HlZjtcMZ...,null,null,null,null,2016-06-24,null,0.508,null,0.0,null,1.0,0.326,0.551,3.41E-6,0.126,0.555,null
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spot...,Argentina,top200,null,153956.0,58IL315gMSTD37DOZ...,null,null,null,null,2016-04-08,null,0.899,null,6.0,null,0.0,0.292,0.076,0.0,0.0631,0.873,null
5,Traicionera,6,2017-01-01,Sebastian Yatra,https://open.spot...,Argentina,top200,null,151140.0,5J1c3M4EldCfNxXwr...,null,null,null,null,2016-06-24,null,0.776,null,11.0,null,1.0,0.0638,0.142,0.0,0.219,0.661,null
6,Cuando Se Pone a ...,7,2017-01-01,Rombai,https://open.spot...,Argentina,top200,null,148369.0,1MpKZi1zTXpERKwxm...,null,null,null,null,2016-12-02,null,0.588,null,11.0,null,0.0,0.173,0.0851,2.69E-5,0.084,0.937,null
7,Otra vez (feat. J...,8,2017-01-01,Zion & Lennox,https://open.spot...,Argentina,top200,null,143004.0,3QwBODjSEzelZyVjx...,null,null,null,null,2016-09-30,null,0.832,null,10.0,null,1.0,0.1,0.0559,4.86E-4,0.44,0.704,null
8,La Bicicleta,9,2017-01-01,"Carlos Vives, Sha...",https://open.spot...,Argentina,top200,null,126389.0,0sXvAOmXgjR2QUqLK...,null,null,null,null,2017-05-26,null,0.736,null,0.0,null,1.0,0.129,0.198,2.32E-6,0.336,0.953,null


In [5]:
data.summary()

DataFrame[summary: string, _c0: string, title: string, rank: string, artist: string, url: string, region: string, chart: string, trend: string, streams: string, track_id: string, album: string, popularity: string, duration_ms: string, available_markets: string, af_danceability: string, af_energy: string, af_key: string, af_loudness: string, af_mode: string, af_speechiness: string, af_acousticness: string, af_instrumentalness: string, af_liveness: string, af_valence: string, af_tempo: string, af_time_signature: string]

In [6]:
data.describe()
#.show(vertical = True)

DataFrame[summary: string, _c0: string, title: string, rank: string, artist: string, url: string, region: string, chart: string, trend: string, streams: string, track_id: string, album: string, popularity: string, duration_ms: string, available_markets: string, af_danceability: string, af_energy: string, af_key: string, af_loudness: string, af_mode: string, af_speechiness: string, af_acousticness: string, af_instrumentalness: string, af_liveness: string, af_valence: string, af_tempo: string, af_time_signature: string]

In [45]:
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- album: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- release_date: date (nullable = true)
 |-- available_markets: string (nullable = true)
 |-- af_danceability: float (nullable = true)
 |-- af_energy: float (nullable = true)
 |-- af_key: integer (nullable = true)
 |-- af_loudness: float (nullable = true)
 |-- af_mode: integer (nullable = true)
 |-- af_speechiness: float (nullable = true)
 |-- af_acousticness: float (nullable = true)
 |-- af_instrumentalness: float (nullable = true)
 |-- af_liveness

In [46]:
data.count()

10000

In [62]:
data.distinct().show()

+----+--------------------+----+----------+--------------------+--------------------+------------------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
| _c0|               title|rank|    artist|                 url|              region|             chart| trend|streams|track_id|               album|popularity|duration_ms|explicit|release_date|available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|
+----+--------------------+----+----------+--------------------+--------------------+------------------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+-

In [48]:
data.dtypes

[('_c0', 'int'),
 ('title', 'string'),
 ('rank', 'int'),
 ('artist', 'string'),
 ('url', 'string'),
 ('region', 'string'),
 ('chart', 'string'),
 ('trend', 'string'),
 ('streams', 'int'),
 ('track_id', 'string'),
 ('album', 'string'),
 ('popularity', 'int'),
 ('duration_ms', 'int'),
 ('explicit', 'boolean'),
 ('release_date', 'date'),
 ('available_markets', 'string'),
 ('af_danceability', 'float'),
 ('af_energy', 'float'),
 ('af_key', 'int'),
 ('af_loudness', 'float'),
 ('af_mode', 'int'),
 ('af_speechiness', 'float'),
 ('af_acousticness', 'float'),
 ('af_instrumentalness', 'float'),
 ('af_liveness', 'float'),
 ('af_valence', 'float'),
 ('af_tempo', 'float'),
 ('af_time_signature', 'int')]

In [53]:
data.stat.freqItems(['title']).show()

+--------------------+
|     title_freqItems|
+--------------------+
|[Mi tesoro (feat....|
+--------------------+



In [57]:
data.select('af_energy').show()

+---------+
|af_energy|
+---------+
|     null|
|    0.852|
|    0.663|
|    0.761|
|    0.508|
|    0.899|
|    0.776|
|    0.588|
|    0.832|
|    0.736|
|    0.721|
|     0.68|
|     0.78|
|     0.73|
|    0.723|
|     0.76|
|    0.598|
|    0.716|
|    0.476|
|    0.549|
+---------+
only showing top 20 rows



In [58]:
data.filter(data['af_energy']>0.5).show()

+---+--------------------+----+----------+--------------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
|_c0|               title|rank|    artist|                 url|              region|    chart| trend|streams|track_id|               album|popularity|duration_ms|explicit|release_date|available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|
+---+--------------------+----+----------+--------------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+

In [64]:
data.select(['title','album','artist']).orderBy(desc('artist')).show()

+----------+--------------------+----------+
|     title|               album|    artist|
+----------+--------------------+----------+
|     title|            track_id|      date|
|    Dueles|1iRvhKiXRElIH2Uf4...|2018-03-02|
|     BILLY|7bRRHQelMfrP86QYQ...|2018-03-02|
|      Mine|6tHWl8ows5JOZq9Yf...|2018-03-01|
|      Mine|6tHWl8ows5JOZq9Yf...|2018-03-01|
|   Outcast|6jnDfM8vtRWgN7R4X...|2018-03-01|
|        Du|0EM85Qj3526QtxCBO...|2018-03-01|
|     Leika|6sIgHBjmtF5AgGa0a...|2018-03-01|
|  Everyday|2zJJro5CPAVwwzLQQ...|2018-03-01|
|      Mine|6tHWl8ows5JOZq9Yf...|2018-03-01|
|   Skyldig|4k9zexSmoSS6EER7F...|2018-03-01|
| STAY TUNE|0albLGvYx6ftOUJ7K...|2018-03-01|
|     BILLY|7bRRHQelMfrP86QYQ...|2018-03-01|
| Ko Ko Bop|5EzitieoPnjyKHAq0...|2018-03-01|
|        楓|73xYYths9kvmFvYsd...|2018-03-01|
|        TT|1nMbZ9OsVNSLEyijI...|2018-03-01|
|   44 More|6iyp6udVpLuq3MCUu...|2018-03-01|
|God's Plan|2XW4DbS6NddZxRPm5...|2018-03-01|
|   Enkelit|6hAKGURpSwL2tvmfj...|2018-03-01|
|      Chic

In [ ]:
spark.sql(f"SELECT * FROM merged_data_10k.csv")

In [71]:
data.show(n=5)

+----+--------------------+----+----------+--------------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
| _c0|               title|rank|    artist|                 url|              region|    chart| trend|streams|track_id|               album|popularity|duration_ms|explicit|release_date|available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|
+----+--------------------+----+----------+--------------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+-------------

In [9]:
data=data.withColumnRenamed("_c0", "index")
  

# Create/Update features

In [ ]:
 data = data.withColumn(

In [95]:
data.select([ 'available_markets','af_danceability','af_energy','af_key','af_loudness','af_mode','af_speechiness', 'af_acousticness','af_instrumentalness','af_liveness','af_valence','af_tempo','af_time_signature']).show()

+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
|available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|
+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+
|     release_date|           null|     null|  null|       null|   null|          null|           null|               null|       null|      null|    null|             null|
|       2017-05-26|           null|    0.852|  null|        8.0|   null|           0.0|         0.0776|              0.187|    3.05E-5|     0.159|   0.907|             null|
|       2016-09-22|           null|    0.663|  null|       11.0|   null|           0.0|          0.226|            0.00431|    1.6

In [104]:
sum(col('af_danceability')).

Column<'sum(af_danceability)'>

In [66]:
data = data.withColumn("title", lower(col('title')))\
            .withColumn("Class", when(col('af_energy')>0.5, "High Energy").otherwise(when(col('af_loudness')>0.5, "Loud").otherwise("Classical")))\
            .withColumn("Weight", when((col('af_loudness').isNotNull()) & (col('af_energy').isNotNull()), col('af_loudness')+ col('af_energy')))\
           

In [111]:
data.show(n=3)

+----+--------------------+----+----------+------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+
| _c0|               title|rank|    artist|         url|              region|    chart| trend|streams|track_id|               album|popularity|duration_ms|explicit|release_date|available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|      Class|Weight|
+----+--------------------+----+----------+------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------

# Fill in missing values

In [40]:
data =data.fillna(0)

In [113]:
data.describe().show(vertical=True)

-RECORD 0-----------------------------------
 summary             | count                
 _c0                 | 10000                
 title               | 10000                
 rank                | 10000                
 artist              | 10000                
 url                 | 10000                
 region              | 10000                
 chart               | 10000                
 trend               | 10000                
 streams             | 10000                
 track_id            | 10000                
 album               | 10000                
 popularity          | 10000                
 duration_ms         | 10000                
 available_markets   | 10000                
 af_danceability     | 10000                
 af_energy           | 10000                
 af_key              | 10000                
 af_loudness         | 10000                
 af_mode             | 10000                
 af_speechiness      | 10000                
 af_acoust

In [114]:
data.show(n=3)

+---+--------------------+----+----------+------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+-----------+------+
|_c0|               title|rank|    artist|         url|              region|    chart| trend|streams|track_id|               album|popularity|duration_ms|explicit|release_date|available_markets|af_danceability|af_energy|af_key|af_loudness|af_mode|af_speechiness|af_acousticness|af_instrumentalness|af_liveness|af_valence|af_tempo|af_time_signature|      Class|Weight|
+---+--------------------+----+----------+------------+--------------------+---------+------+-------+--------+--------------------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+--

In [ ]:
import os
import airflow

In [50]:
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile","/home/rihab/Downloads/golden-system-411709-d84051b4e799.json") 

In [32]:
spark.stop()

In [84]:
hadoop_version = spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion()
print(f"Hadoop Version: {hadoop_version}")


Hadoop Version: 3.3.2


In [3]:
client = storage.Client.from_service_account_json('/home/rihab/Downloads/golden-system-411709-d84051b4e799.json')

In [6]:
gcs_bucket = "spotify_data_rg_01"
file_name = "spotify.csv"

In [5]:
bucket=client.get_bucket(gcs_bucket)

In [7]:
blob= bucket.blob(file_name)

In [13]:
data.coalesce(1).write.csv('/tmp/data.csv', header=True, mode="overwrite")

In [15]:
import glob
file_path = glob.glob('/tmp/data.csv' + "/*.csv")[0]
blob.upload_from_filename(file_path)

In [11]:
blob.upload_from_file(data)

AttributeError: 'DataFrame' object has no attribute 'tell'

In [101]:

file_path = f"gs://{gcs_bucket}/{file_name}/"
data = data.coalesce(1)
data.write.csv(file_path, header=True)

Py4JJavaError: An error occurred while calling o1096.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:461)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:558)
	at org.apache.spark.sql.DataFrameWriter.saveToV1

In [ ]:
def read_files_in_batches(directory, batch_size):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    for i in range(0, len(files), batch_size):
        yield files[i:i+batch_size]

In [ ]:
for file in read_files_in_batches('../data',2):
    print(file)